In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from glob import glob
import os
import re
import pyemma.msm as msm
import pyemma.coordinates as coor
import numpy as np
import my_network_plot as myplt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import ticker
import operator
import pandas as pd
import mdtraj as md
import matplotlib.image as mpimg 
import pickle
from shutil import move

## Helper functions

In [35]:
import numpy.linalg as la
lag_ps = 10
def get_rate_matrix(T):
    # T: transition matrix
    # K: rate matrix
    # diagnonalize
    w, v = la.eig(T)
    # sort eigenvectors and eigenvalues
    order = np.argsort(w)[::-1]
    w = w[order]
    v = v[:, order]
    # create diagonal matrix
    n = len(w)
    ln_T_diag = np.eye(n)
    ln_T_diag[np.eye(n, dtype=bool)] =  np.log(w)
    # convert to original basis
    lnT = la.multi_dot((v,ln_T_diag, la.inv(v)))
    K = lnT/lag_ps
    return K

In [36]:
def get_timescales(T):
    # Get all the  timescales for non-zero rates
    # T: transition matrix
    # ts: timescales
    K = get_rate_matrix(mm.sample_mean('transition_matrix'))
    off_diag = K[~np.eye(K.shape[0], dtype=bool)]
    ts = 1/off_diag[off_diag>0]
    return ts


## Check for negative eigenvalues

In [16]:

bhmm_paths = glob('bhmm_mods/*.p')
bhmm_paths[:10]
# for i in range(len(bayes_mods['traj_num'])):
#     f_name = 'traj_{0}-idx_{1}_bhmm.p'.format(bayes_mods['traj_num'][i], bayes_mods['time_idx'][i])
#     pickle.dump(obj=bayes_mods['mod'][i], file=open(f_name, 'wb'))

['bhmm_mods/traj_7-idx_157_bhmm.p',
 'bhmm_mods/traj_7-idx_813_bhmm.p',
 'bhmm_mods/traj_1-idx_519_bhmm.p',
 'bhmm_mods/traj_8-idx_254_bhmm.p',
 'bhmm_mods/traj_9-idx_786_bhmm.p',
 'bhmm_mods/traj_9-idx_360_bhmm.p',
 'bhmm_mods/traj_9-idx_431_bhmm.p',
 'bhmm_mods/traj_6-idx_163_bhmm.p',
 'bhmm_mods/traj_6-idx_632_bhmm.p',
 'bhmm_mods/traj_1-idx_21_bhmm.p']

In [17]:
for path in bhmm_paths:
    print(path)
    mm = pickle.load(open(path, 'rb'))
    eigs = mm.sample_mean('eigenvalues')
    if np.any(eigs<0):
        print('\t moving')
        move(path, path.replace('bhmm_mods', 'bhmm_mods/negative_evs'))

bhmm_mods/traj_7-idx_157_bhmm.p
bhmm_mods/traj_7-idx_813_bhmm.p
bhmm_mods/traj_1-idx_519_bhmm.p
bhmm_mods/traj_8-idx_254_bhmm.p
bhmm_mods/traj_9-idx_786_bhmm.p
bhmm_mods/traj_9-idx_360_bhmm.p
bhmm_mods/traj_9-idx_431_bhmm.p
bhmm_mods/traj_6-idx_163_bhmm.p
bhmm_mods/traj_6-idx_632_bhmm.p
bhmm_mods/traj_1-idx_21_bhmm.p
bhmm_mods/traj_6-idx_651_bhmm.p
bhmm_mods/traj_7-idx_149_bhmm.p
bhmm_mods/traj_3-idx_771_bhmm.p
bhmm_mods/traj_4-idx_790_bhmm.p
bhmm_mods/traj_8-idx_180_bhmm.p
bhmm_mods/traj_4-idx_527_bhmm.p
bhmm_mods/traj_6-idx_839_bhmm.p
bhmm_mods/traj_7-idx_134_bhmm.p
bhmm_mods/traj_10-idx_41_bhmm.p
bhmm_mods/traj_7-idx_145_bhmm.p
bhmm_mods/traj_3-idx_151_bhmm.p
bhmm_mods/traj_9-idx_69_bhmm.p
bhmm_mods/traj_8-idx_246_bhmm.p
bhmm_mods/traj_9-idx_272_bhmm.p
bhmm_mods/traj_9-idx_881_bhmm.p
bhmm_mods/traj_6-idx_720_bhmm.p
bhmm_mods/traj_4-idx_219_bhmm.p
bhmm_mods/traj_8-idx_509_bhmm.p
bhmm_mods/traj_8-idx_192_bhmm.p
bhmm_mods/traj_3-idx_80_bhmm.p
bhmm_mods/traj_1-idx_239_bhmm.p
bhmm_mods/t

bhmm_mods/traj_6-idx_862_bhmm.p
bhmm_mods/traj_3-idx_106_bhmm.p
	 moving
bhmm_mods/traj_8-idx_311_bhmm.p
	 moving
bhmm_mods/traj_3-idx_757_bhmm.p
bhmm_mods/traj_7-idx_856_bhmm.p
	 moving
bhmm_mods/traj_10-idx_16_bhmm.p
bhmm_mods/traj_9-idx_358_bhmm.p
bhmm_mods/traj_3-idx_821_bhmm.p
	 moving
bhmm_mods/traj_7-idx_935_bhmm.p
bhmm_mods/traj_3-idx_165_bhmm.p
	 moving
bhmm_mods/traj_4-idx_562_bhmm.p
	 moving
bhmm_mods/traj_6-idx_669_bhmm.p
	 moving
bhmm_mods/traj_8-idx_523_bhmm.p
bhmm_mods/traj_3-idx_734_bhmm.p
	 moving
bhmm_mods/traj_6-idx_901_bhmm.p
	 moving
bhmm_mods/traj_9-idx_246_bhmm.p
bhmm_mods/traj_6-idx_801_bhmm.p
	 moving
bhmm_mods/traj_3-idx_118_bhmm.p
bhmm_mods/traj_6-idx_714_bhmm.p
bhmm_mods/traj_6-idx_614_bhmm.p
bhmm_mods/traj_8-idx_297_bhmm.p
	 moving
bhmm_mods/traj_3-idx_237_bhmm.p
	 moving
bhmm_mods/traj_4-idx_925_bhmm.p
	 moving
bhmm_mods/traj_7-idx_223_bhmm.p
bhmm_mods/traj_7-idx_194_bhmm.p
	 moving
bhmm_mods/traj_7-idx_75_bhmm.p
	 moving
bhmm_mods/traj_6-idx_899_bhmm.p
bh

	 moving
bhmm_mods/traj_4-idx_550_bhmm.p
	 moving
bhmm_mods/traj_4-idx_201_bhmm.p
bhmm_mods/traj_10-idx_642_bhmm.p
bhmm_mods/traj_3-idx_157_bhmm.p
	 moving
bhmm_mods/traj_8-idx_511_bhmm.p
bhmm_mods/traj_3-idx_813_bhmm.p
bhmm_mods/traj_7-idx_712_bhmm.p
	 moving
bhmm_mods/traj_6-idx_151_bhmm.p
bhmm_mods/traj_1-idx_207_bhmm.p
bhmm_mods/traj_6-idx_700_bhmm.p
bhmm_mods/traj_9-idx_252_bhmm.p
bhmm_mods/traj_7-idx_649_bhmm.p
bhmm_mods/traj_9-idx_503_bhmm.p
	 moving
bhmm_mods/traj_6-idx_815_bhmm.p
	 moving
bhmm_mods/traj_4-idx_94_bhmm.p
bhmm_mods/traj_9-idx_352_bhmm.p
bhmm_mods/traj_8-idx_780_bhmm.p
bhmm_mods/traj_8-idx_266_bhmm.p
bhmm_mods/traj_8-idx_537_bhmm.p
	 moving
bhmm_mods/traj_1-idx_989_bhmm.p
	 moving
bhmm_mods/traj_3-idx_935_bhmm.p
	 moving
bhmm_mods/traj_10-idx_971_bhmm.p
	 moving
bhmm_mods/traj_6-idx_868_bhmm.p
bhmm_mods/traj_6-idx_968_bhmm.p
bhmm_mods/traj_7-idx_106_bhmm.p
	 moving
bhmm_mods/traj_10-idx_912_bhmm.p
	 moving
bhmm_mods/traj_3-idx_856_bhmm.p
bhmm_mods/traj_4-idx_415_b

bhmm_mods/traj_9-idx_849_bhmm.p
	 moving
bhmm_mods/traj_3-idx_56_bhmm.p
bhmm_mods/traj_1-idx_158_bhmm.p
	 moving
bhmm_mods/traj_8-idx_615_bhmm.p
bhmm_mods/traj_8-idx_900_bhmm.p
bhmm_mods/traj_8-idx_800_bhmm.p
bhmm_mods/traj_4-idx_941_bhmm.p
bhmm_mods/traj_7-idx_247_bhmm.p
	 moving
bhmm_mods/traj_4-idx_830_bhmm.p
bhmm_mods/traj_8-idx_971_bhmm.p
bhmm_mods/traj_1-idx_129_bhmm.p
bhmm_mods/traj_4-idx_725_bhmm.p
bhmm_mods/traj_3-idx_195_bhmm.p
bhmm_mods/traj_4-idx_592_bhmm.p
bhmm_mods/traj_6-idx_799_bhmm.p
	 moving
bhmm_mods/traj_3-idx_222_bhmm.p
bhmm_mods/traj_6-idx_699_bhmm.p
bhmm_mods/traj_4-idx_70_bhmm.p
	 moving
bhmm_mods/traj_1-idx_154_bhmm.p
	 moving
bhmm_mods/traj_8-idx_73_bhmm.p
	 moving
bhmm_mods/traj_6-idx_553_bhmm.p
	 moving
bhmm_mods/traj_4-idx_658_bhmm.p
	 moving
bhmm_mods/traj_6-idx_787_bhmm.p
bhmm_mods/traj_1-idx_137_bhmm.p
	 moving
bhmm_mods/traj_3-idx_39_bhmm.p
bhmm_mods/traj_1-idx_666_bhmm.p
bhmm_mods/traj_4-idx_13_bhmm.p
	 moving
bhmm_mods/traj_7-idx_228_bhmm.p
bhmm_mods/

	 moving
bhmm_mods/traj_6-idx_963_bhmm.p
	 moving
bhmm_mods/traj_9-idx_324_bhmm.p
	 moving
bhmm_mods/traj_3-idx_298_bhmm.p
bhmm_mods/traj_4-idx_379_bhmm.p
bhmm_mods/traj_6-idx_723_bhmm.p
bhmm_mods/traj_1-idx_2_bhmm.p
bhmm_mods/traj_9-idx_271_bhmm.p
bhmm_mods/traj_9-idx_371_bhmm.p
bhmm_mods/traj_8-idx_514_bhmm.p
bhmm_mods/traj_1-idx_359_bhmm.p
bhmm_mods/traj_7-idx_146_bhmm.p
bhmm_mods/traj_6-idx_828_bhmm.p
bhmm_mods/traj_7-idx_674_bhmm.p
bhmm_mods/traj_3-idx_975_bhmm.p
	 moving
bhmm_mods/traj_6-idx_67_bhmm.p
bhmm_mods/traj_3-idx_131_bhmm.p
bhmm_mods/traj_3-idx_83_bhmm.p
bhmm_mods/traj_6-idx_855_bhmm.p
bhmm_mods/traj_9-idx_543_bhmm.p
	 moving
bhmm_mods/traj_9-idx_212_bhmm.p
bhmm_mods/traj_7-idx_158_bhmm.p
	 moving
bhmm_mods/traj_9-idx_443_bhmm.p
	 moving
bhmm_mods/traj_3-idx_808_bhmm.p
bhmm_mods/traj_1-idx_30_bhmm.p
	 moving
bhmm_mods/traj_6-idx_640_bhmm.p
	 moving
bhmm_mods/traj_10-idx_108_bhmm.p
	 moving
bhmm_mods/traj_6-idx_731_bhmm.p
	 moving
bhmm_mods/traj_6-idx_631_bhmm.p
bhmm_mods

	 moving
bhmm_mods/traj_1-idx_524_bhmm.p
bhmm_mods/traj_9-idx_220_bhmm.p
	 moving
bhmm_mods/traj_3-idx_752_bhmm.p
	 moving
bhmm_mods/traj_3-idx_103_bhmm.p
	 moving
bhmm_mods/traj_3-idx_847_bhmm.p
bhmm_mods/traj_3-idx_836_bhmm.p
bhmm_mods/traj_6-idx_24_bhmm.p
	 moving
bhmm_mods/traj_8-idx_996_bhmm.p
	 moving
bhmm_mods/traj_7-idx_166_bhmm.p
bhmm_mods/traj_1-idx_379_bhmm.p
bhmm_mods/traj_7-idx_922_bhmm.p
	 moving
bhmm_mods/traj_8-idx_534_bhmm.p
	 moving
bhmm_mods/traj_3-idx_723_bhmm.p
	 moving
bhmm_mods/traj_9-idx_351_bhmm.p
bhmm_mods/traj_6-idx_59_bhmm.p
bhmm_mods/traj_1-idx_304_bhmm.p
bhmm_mods/traj_9-idx_37_bhmm.p
bhmm_mods/traj_6-idx_703_bhmm.p
	 moving
bhmm_mods/traj_6-idx_152_bhmm.p
bhmm_mods/traj_6-idx_603_bhmm.p
	 moving
bhmm_mods/traj_1-idx_73_bhmm.p
	 moving
bhmm_mods/traj_6-idx_286_bhmm.p
	 moving
bhmm_mods/traj_6-idx_131_bhmm.p
	 moving
bhmm_mods/traj_6-idx_660_bhmm.p
bhmm_mods/traj_1-idx_267_bhmm.p
bhmm_mods/traj_6-idx_386_bhmm.p
	 moving
bhmm_mods/traj_9-idx_332_bhmm.p
bhmm_

bhmm_mods/traj_6-idx_117_bhmm.p
	 moving
bhmm_mods/traj_6-idx_646_bhmm.p
bhmm_mods/traj_8-idx_220_bhmm.p
bhmm_mods/traj_3-idx_137_bhmm.p
bhmm_mods/traj_8-idx_197_bhmm.p
	 moving
bhmm_mods/traj_3-idx_85_bhmm.p
	 moving
bhmm_mods/traj_7-idx_123_bhmm.p
	 moving
bhmm_mods/traj_9-idx_269_bhmm.p
bhmm_mods/traj_9-idx_538_bhmm.p
bhmm_mods/traj_3-idx_873_bhmm.p
	 moving
bhmm_mods/traj_9-idx_438_bhmm.p
	 moving
bhmm_mods/traj_10-idx_27_bhmm.p
bhmm_mods/traj_7-idx_140_bhmm.p
bhmm_mods/traj_3-idx_810_bhmm.p
	 moving
bhmm_mods/traj_1-idx_28_bhmm.p
bhmm_mods/traj_8-idx_243_bhmm.p
bhmm_mods/traj_6-idx_658_bhmm.p
bhmm_mods/traj_8-idx_512_bhmm.p
bhmm_mods/traj_4-idx_202_bhmm.p
	 moving
bhmm_mods/traj_9-idx_791_bhmm.p
bhmm_mods/traj_1-idx_4_bhmm.p
bhmm_mods/traj_9-idx_377_bhmm.p
	 moving
bhmm_mods/traj_9-idx_426_bhmm.p
bhmm_mods/traj_3-idx_778_bhmm.p
	 moving
bhmm_mods/traj_1-idx_55_bhmm.p
bhmm_mods/traj_6-idx_174_bhmm.p
	 moving
bhmm_mods/traj_6-idx_625_bhmm.p
	 moving
bhmm_mods/traj_8-idx_189_bhmm.p
b

# Get timescales

In [49]:
timescales = {'traj_idx': [], 'time_idx': [], 'timescales': [], 'n_states': [], 'n_timescales': []}
bhmm_paths = glob('bhmm_mods/*.p')
for path in bhmm_paths:
    traj_idx = re.findall('[0-9]+', path)[0]
    time_idx = re.findall('[0-9]+', path)[1]
    
    mm = pickle.load(open(path, 'rb'))
    move(path, path.replace('bhmm_mods', 'bhmm_mods/{}_state'.format(mm.nstates)))
    
    T = mm.sample_mean('transition_matrix')
    ts = get_timescales(T)
    n_ts = ts.shape[0]
    timescales['traj_idx'].extend(np.repeat(traj_idx, n_ts))
    timescales['time_idx'].extend(np.repeat(time_idx, n_ts))
    timescales['timescales'].extend(ts)
    timescales['n_states'].extend(np.repeat(mm.nstates, n_ts))
    timescales['n_timescales'].extend(np.repeat(n_ts, n_ts))
    print(traj_idx, time_idx, n_ts, mm.nstates)
    

7 157 2 2
7 813 2 2
1 519 2 2
8 254 2 2
9 786 2 2
9 360 6 3
9 431 2 2
6 163 2 2
6 632 2 2
1 21 2 2
6 651 2 2
7 149 2 2
3 771 2 2
4 790 2 2
8 180 2 2
4 527 2 2
6 839 2 2
7 134 2 2
10 41 2 2
7 145 2 2
3 151 2 2
9 69 2 2
8 246 2 2
9 272 4 3
9 881 2 2
6 720 2 2
4 219 2 2
8 509 2 2
8 192 2 2
3 80 2 2
1 239 2 2
3 736 2 2
9 239 2 2
1 540 2 2
6 903 2 2
9 344 2 2
1 881 2 2
1 523 2 2
6 708 4 3
3 755 2 2
3 724 2 2
6 779 2 2
4 223 2 2
6 155 2 2
1 74 2 2
3 108 2 2
7 958 2 2
6 704 2 2
9 256 2 2
6 811 2 2
9 356 2 2
9 564 2 2
1 993 2 2
7 599 2 2
1 360 2 2
6 767 2 2
7 653 2 2
9 519 2 2
7 102 2 2
9 348 2 2
8 550 2 2
3 116 6 3
7 34 2 2
3 985 4 3
3 885 2 2
4 671 2 2
4 8 2 2
7 49 2 2
9 604 2 2
4 24 2 2
6 564 6 3
3 215 2 2
6 419 2 2
8 946 2 2
4 807 2 2
7 201 2 2
3 997 2 2
4 285 2 2
4 763 2 2
6 568 2 2
3 782 2 2
1 112 2 2
8 35 2 2
9 860 2 2
3 889 2 2
8 929 2 2
7 38 2 2
9 159 2 2
1 848 2 2
8 954 2 2
1 159 2 2
4 655 2 2
1 99 2 2
6 272 2 2
6 694 2 2
9 856 2 2
1 147 2 2
6 211 2 2
7 225 2 2
3 45 4 3
6 431 2 2
1 7

/Users/robertarbon/miniconda3/envs/science/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in log
  from ipykernel import kernelapp as app
/Users/robertarbon/miniconda3/envs/science/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in greater
  import sys


9 304 0 3
9 204 2 2
9 62 2 2
1 26 2 2
9 428 2 2
7 384 2 2
9 279 2 2
3 963 2 2
3 127 2 2
10 643 2 2
10 112 2 2
7 906 2 2
3 812 2 2
9 308 2 2
6 627 2 2
6 832 2 2
3 871 2 2
9 998 2 2
4 785 2 2
8 195 2 2
8 984 2 2
3 824 2 2
7 592 2 2
3 160 2 2
9 243 2 2
1 61 2 2
1 524 2 2
3 847 2 2
3 836 2 2
7 166 2 2
1 379 2 2
9 351 2 2
6 59 2 2
1 304 2 2
9 37 2 2
6 152 2 2
6 660 2 2
1 267 2 2
9 332 2 2
8 557 2 2
7 105 2 2
10 335 2 2
10 235 2 2
7 896 2 2
7 265 2 2
3 882 2 2
7 33 2 2
6 251 2 2
9 916 2 2
6 400 2 2
9 131 2 2
9 760 2 2
8 178 2 2
4 768 2 2
6 563 2 2
3 789 2 2
4 715 2 2
1 119 2 2
3 785 2 2
8 592 2 2
9 879 2 2
3 990 2 2
3 890 2 2
3 3 2 2
7 438 2 2
9 294 2 2
6 571 2 2
10 339 2 2
3 78 2 2
4 812 2 2
7 510 2 2
4 103 2 2
3 255 2 2
8 613 2 2
6 559 2 2
6 693 2 2
1 123 2 2
6 793 2 2
8 918 2 2
10 794 2 2
1 879 2 2
1 908 2 2
9 635 2 2
1 875 2 2
6 304 2 2
3 259 2 2
1 916 2 2
3 21 2 2
6 279 2 2
4 594 2 2
3 224 2 2
10 993 2 2
7 187 2 2
6 473 2 2
4 678 2 2
3 799 2 2
6 195 2 2
8 53 2 2
3 699 2 2
1 830 2 2
3 20

In [51]:
pd.DataFrame(timescales).to_csv('bhmm_mods/bhmm_mods_stats.csv', index_label=False, index=False)

## Free energy barriers 

$$
k=\kappa {\frac {k_{B}T}{h}}e^{\frac {\Delta S^{\ddagger }}{R}}e^{\frac {-\Delta H^{\ddagger }}{RT}}
$$

In [ ]:
from scipy.constants import k, h, R, calorie

T = 300
ps_to_sec = 10**12

def ts_to_dg(ts):
    return -R*T*np.log(h/(ts*k*T))/1000/calorie

In [ ]:
df['mean_ts'] = df['mean_ts_ps']/ps_to_sec
df['dg'] = df['mean_ts'].apply(ts_to_dg)

In [ ]:
df.head()

In [ ]:
with sns.plotting_context('paper', font_scale=2):
    ax = sns.distplot(df['dg'], hist=True, kde=False, norm_hist=False)
    _, ub = ax.get_xlim()
    ax.set_xlim(0, ub)
    ax.set_ylabel('Frequency')
    ax.set_xlabel('Free energy barrier ($\mathrm{kcal}\cdot\mathrm{mol^{-1}}$)')
    plt.savefig('free_energy_barrier_hist.png', dpi=600, bbox_inches='tight')

In [ ]:
with sns.plotting_context('paper', font_scale=2):
    g = sns.FacetGrid(data=df, col='traj_num', col_wrap=4)
    g.map(sns.distplot, 'dg', hist=True, kde=False, norm_hist=False)
    g.set_titles("Trajectory {col_name}")
    for ax in g.axes.flatten():
        _, ub = ax.get_xlim()
        ax.set_xlim(0, ub)
    _ = [g.axes[i].set_ylabel('Frequency') for i in [0, 4]]
    _ = [g.axes[i].set_xlabel('Free energy barrier\n($\mathrm{kcal}\cdot\mathrm{mol^{-1}}$)') for i in range(4,8)]
    plt.savefig('free_energy_barrier_hist_by_traj.png', dpi=600, bbox_inches='tight')

In [ ]:
# df = pd.read_csv('all_timescales.csv')
